### МОДЕЛИ КЛАССИФИКАЦИИ ДЛЯ ПРОГНОЗИРОВАНИЯ ОТКЛИКА

In [ ]:
# Импорт необходимых библиотек для классификации
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             roc_auc_score, confusion_matrix, classification_report,
                             roc_curve, precision_recall_curve)
import joblib
import pickle
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from collections import Counter

In [ ]:
df = pd.read_csv('../data/marketing_campaign.csv', sep='\t')

# Нормализация категорий
df["Education"] = df["Education"].replace({"2n Cycle": "Pre-Graduate", "Basic": "Pre-Graduate"})
df["Marital_Status"] = df["Marital_Status"].replace({
    "Married": "Married/Together", "Together": "Married/Together",
    "Single": "Single", "Divorced": "Other", "Widow": "Other",
    "Alone": "Other", "Absurd": "Other", "YOLO": "Other"
})

# Feature engineering
df["Kids"] = df["Kidhome"].astype("int8") + df["Teenhome"].astype("int8")
df["Expenses"] = df[["MntWines", "MntFruits", "MntMeatProducts", "MntFishProducts", "MntSweetProducts", "MntGoldProds"]].sum(axis=1)
df["TotalAcceptedCmp"] = df[["AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", "AcceptedCmp4", "AcceptedCmp5"]].astype("int8").sum(axis=1)
df["TotalNumPurchases"] = df[["NumWebPurchases", "NumCatalogPurchases", "NumStorePurchases", "NumDealsPurchases"]].sum(axis=1)

# Удаление лишних колонок
df.drop(columns=["Kidhome", "Teenhome", "MntWines", "MntFruits", "MntMeatProducts", "MntFishProducts", 
                "MntSweetProducts", "MntGoldProds", "AcceptedCmp1", "AcceptedCmp2", "AcceptedCmp3", 
                "AcceptedCmp4", "AcceptedCmp5", "NumWebPurchases", "NumCatalogPurchases", 
                "NumStorePurchases", "NumDealsPurchases"], inplace=True)

df["Kids"] = df["Kids"].replace({0: "No Kid", 1: "Has Kids", 2: "Has Kids", 3: "Has Kids"})
df["TotalAcceptedCmp"] = df["TotalAcceptedCmp"].replace({0: "0", 1: ">0", 2: ">0", 3: ">0", 4: ">0"})

num_features = ["Income", "Recency", "NumWebVisitsMonth", "Expenses", "TotalNumPurchases"]
cat_features = ["Education", "Marital_Status", "Response", "Complain", "Kids", "TotalAcceptedCmp"]

# Очистка Income
df['Income'] = df['Income'].fillna(df['Income'].median())
df = df[df['Income'] < 600000]

In [ ]:
# Определяем признаки для модели
# Числовые признаки 
numeric_features = ["Income", "Recency", "NumWebVisitsMonth", "Expenses", "TotalNumPurchases"]

# Категориальные признаки 
categorical_features = ["Education", "Marital_Status", "Complain", "Kids", "TotalAcceptedCmp"]

# Выбираем все признаки для модели
feature_cols = numeric_features + categorical_features 

In [ ]:
# Создаем матрицу признаков и целевую переменную
X = df[feature_cols].copy()
y = df['Response'].copy()

In [ ]:
# Визуализация дисбаланса классов
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
# Pie chart
axes[0].pie(y.value_counts(), labels=['Нет отклика (0)', 'Отклик (1)'], 
            autopct='%1.1f%%', colors=['#ff9999', '#66b3ff'], startangle=90)
axes[0].set_title('Распределение целевой переменной', fontsize=14)
# Bar chart
axes[1].bar(['Нет отклика (0)', 'Отклик (1)'], y.value_counts(), 
            color=['#ff9999', '#66b3ff'], edgecolor='black')
axes[1].set_title('Количество наблюдений по классам', fontsize=14)
axes[1].set_ylabel('Количество')

plt.tight_layout()
plt.show()

In [ ]:
# Подготовка данных для классификации
# Целевая переменная - Response (отклик на последнюю кампанию)
print("Распределение целевой переменной:")
print(df['Response'].value_counts())

In [ ]:
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

In [ ]:
# Создаем препроцессор для обработки разных типов данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_features)
    ]
)

# Обучаем препроцессор на тренировочных данных
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Получаем названия признаков после обработки
# Для числовых - оставляем оригинальные названия
num_feature_names = numeric_features

# Для категориальных - получаем названия после one-hot
cat_feature_names = []
for cat_feat in categorical_features:
    categories = preprocessor.named_transformers_['cat'].categories_[categorical_features.index(cat_feat)]
    for cat in categories:
        cat_feature_names.append(f"{cat_feat}_{cat}")
        
all_feature_names = num_feature_names + cat_feature_names

print(f"Всего признаков после обработки: {len(all_feature_names)}")

# Сохраняем препроцессор
joblib.dump(preprocessor, 'preprocessor.pkl')

### МЕТОД 1: ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ

In [ ]:
# Логистическая регрессия с подбором гиперпараметров
log_reg_params = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'],
    'solver': ['lbfgs', 'liblinear'],
    'class_weight': [None, 'balanced']
}

log_reg = LogisticRegression(random_state=42, max_iter=1000)

# Grid Search с кросс-валидацией
log_reg_grid = GridSearchCV(
    log_reg, 
    log_reg_params, 
    cv=StratifiedKFold(5, shuffle=True, random_state=42),
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

log_reg_grid.fit(X_train_processed, y_train)

print("\nЛучшие параметры для логистической регрессии:")
print(log_reg_grid.best_params_)
print(f"Лучший ROC-AUC на кросс-валидации: {log_reg_grid.best_score_:.4f}")

# Лучшая модель
best_log_reg = log_reg_grid.best_estimator_

# Предсказания
y_pred_log_reg = best_log_reg.predict(X_test_processed)
y_pred_proba_log_reg = best_log_reg.predict_proba(X_test_processed)[:, 1]

### МЕТОД 2: СЛУЧАЙНЫЙ ЛЕС (Random Forest)

In [ ]:
# Random Forest с подбором гиперпараметров
rf_params = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'class_weight': [None, 'balanced']
}

rf = RandomForestClassifier(random_state=42, n_jobs=-1)

# Grid Search с кросс-валидацией
rf_grid = GridSearchCV(
    rf, 
    rf_params, 
    cv=StratifiedKFold(5, shuffle=True, random_state=42),
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

rf_grid.fit(X_train_processed, y_train)

print("\nЛучшие параметры для Random Forest:")
print(rf_grid.best_params_)
print(f"Лучший ROC-AUC на кросс-валидации: {rf_grid.best_score_:.4f}")

# Лучшая модель
best_rf = rf_grid.best_estimator_

# Предсказания
y_pred_rf = best_rf.predict(X_test_processed)
y_pred_proba_rf = best_rf.predict_proba(X_test_processed)[:, 1]

### МЕТОД 3: GRADIENT BOOSTING

In [ ]:
# Gradient Boosting с подбором гиперпараметров
gb_params = {
    'n_estimators': [100, 200],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'min_samples_split': [2, 5]
}

gb = GradientBoostingClassifier(random_state=42)

# Grid Search с кросс-валидацией
gb_grid = GridSearchCV(
    gb, 
    gb_params, 
    cv=StratifiedKFold(5, shuffle=True, random_state=42),
    scoring='roc_auc',
    n_jobs=-1,
    verbose=1
)

gb_grid.fit(X_train_processed, y_train)

print("\nЛучшие параметры для Gradient Boosting:")
print(gb_grid.best_params_)
print(f"Лучший ROC-AUC на кросс-валидации: {gb_grid.best_score_:.4f}")

# Лучшая модель
best_gb = gb_grid.best_estimator_

# Предсказания
y_pred_gb = best_gb.predict(X_test_processed)
y_pred_proba_gb = best_gb.predict_proba(X_test_processed)[:, 1]